# Probabilistic Data Structure for Mining of Massive Dataset
- [resources](https://www.safaribooksonline.com/oriole/probabilistic-data-structures-in-python)

## What it is

### Probabilistic Data Strucuture usually for Approximation Algorithms

### Examples of algorithms (ordered in complexity progress?)
<table>
<tr>
    <th>Algorithm</th>
    <th>usage</th>
</tr>
<tr>
    <td>HyperLogLog</td><td>set cardinality</td>
</tr>
<tr>
    <td>BloomFilter</td><td>set membership</td>
</tr>
<tr>
    <td>MinHash</td><td>set similiarity</td>
</tr>
<tr>
    <td>Count-Min Sketch</td><td>frequencyt summary</td>
</tr>
<tr>
    <td>t-Digest</td><td>streaming quantiles</td>
</tr>
</table>

In [1]:
jabber_text = """
`Twas brillig, and the slithy toves
  Did gyre and gimble in the wabe:
All mimsy were the borogoves,
  And the mome raths outgrabe.

"Beware the Jabberwock, my son!
  The jaws that bite, the claws that catch!
Beware the Jubjub bird, and shun
  The frumious Bandersnatch!"
He took his vorpal sword in hand:
  Long time the manxome foe he sought --
So rested he by the Tumtum tree,
  And stood awhile in thought.
And, as in uffish thought he stood,
  The Jabberwock, with eyes of flame,
Came whiffling through the tulgey wood,
  And burbled as it came!
One, two! One, two! And through and through
  The vorpal blade went snicker-snack!
He left it dead, and with its head
  He went galumphing back.
"And, has thou slain the Jabberwock?
  Come to my arms, my beamish boy!
O frabjous day! Callooh! Callay!'
  He chortled in his joy.

`Twas brillig, and the slithy toves
  Did gyre and gimble in the wabe;
All mimsy were the borogoves,
  And the mome raths outgrabe.
"""

parker_text = """
My answers are inadequate
To those demanding day and date
And ever set a tiny shock
Through strangers asking what's o'clock;
Whose days are spent in whittling rhyme-
What's time to her, or she to Time?
"""

import re
pat_word = re.compile(r"\w+")
jabber_words = [w.lower() for w in pat_word.findall(jabber_text)]
parker_words = [w.lower() for w in pat_word.findall(parker_text)]

## HyperLogLog for counting unique elements
- [github](https://github.com/svpcom/hyperloglog)
- [wiki](https://en.wikipedia.org/wiki/HyperLogLog)
- key idea: use hash to distribute items to uniform distribution, and estimate the set size with max leading zeros in the binary representation
- usage:
    - set the acceptable error and do the approximation

In [2]:
import hyperloglog

# accept error rate of 1%
hll = hyperloglog.HyperLogLog(0.01)

for word in jabber_words:
    hll.add(word)
    
print ("hll set size vs real size", len(hll), len(set(jabber_words)))

hll set size vs real size 90 91


## Bloomfilter for searching a set
- [github](https://github.com/jaybaird/python-bloomfilter)
- [wiki](https://en.wikipedia.org/wiki/Bloom_filter)
- key idea
    - use k hash function to hash elements into the same hash array
    - check if an element is in hash by checking all k bits are on
    - there won't be false negative
    - but it could have false positive when all bits are turned on by other elements
    - it is a quick way to implement cashing before an expensive database query
    - it is O(n) to implement "intersection" of two sets

*** pybloom is currently only supported on python2.x so I change to `bloom_filter`***

In [3]:
## find the intersection of jabber_words and parker_words

from bloom_filter import BloomFilter

bf = BloomFilter(max_elements=10000, error_rate=0.01)

for w in jabber_words:
    bf.add(w)
    
common = set()
for w in parker_words:
    if w in bf:
        common.add(w)
        
print("estimate vs real intersection size", len(common), len(set(jabber_words) & set(parker_words)))

estimate vs real intersection size 8 8


## Min-hash: Min-wise independent permutations (locality sensitive hashing) for Jaccard
- [github](https://github.com/ekzhu/datasketch)
- [wiki](https://en.wikipedia.org/wiki/MinHash)
- it approximates Jaccard similiarity
    $J = \frac {|A \cap B|}{|A \cup B|}$
- it also uses k hash functions
- usage:
    - it calculates `digest` (figerprint) of sets
    - based on figerprints the similarity can be calculated

In [4]:
from hashlib import sha1
from datasketch import MinHash

def mh_digest(s):
    m = MinHash(num_perm=512) # memory allocation
    for x in s:
        m.update(x.encode('utf8'))
    return m

jabber_digest = mh_digest(jabber_words)
parker_digest = mh_digest(parker_words)

inter = len(set(jabber_words) & set(parker_words))
union = len(set(jabber_words) | set(parker_words))

print("estimated vs actual", jabber_digest.jaccard(parker_digest),
                            inter / union)

estimated vs actual 0.060546875 0.06956521739130435


In [5]:
s = 'hello'
type(s), type(s.encode('utf-8'))

(str, bytes)

## count-min sketch for frequency count
- [github](https://github.com/IsaacHaze/countminsketch)
- [wiki](https://en.wikipedia.org/wiki/Count%E2%80%93min_sketch)
- key idea: 
    - similiar data structure (k hash) as used in bloom filter
    - it is like a frequency table, even for streaming data, so conceptually it should look like a Counter in python

In [8]:
from collections import Counter
import countminsketch

SyntaxError: invalid syntax (countminsketch.py, line 56)

## t-digest for streaming quantiles
- [github](https://github.com/trademob/t-digest)
- [wiki](https://github.com/tdunning/t-digest/blob/master/docs/t-digest-paper/histo.pdf)

In [24]:
## generate random numbers and look at its head, median and tail
from tdigest import TDigest
import random

random.seed(314)

td = TDigest()

xs = []

for _ in range(1000):
    x = random.random()
    td.add(x, 1) # put weight as 1
    xs.append(x)
    
xs = np.asarray(xs)
    
for q in [0.05, 0.5, 0.95]:
    print(q, "estimate vs real", td.quantile(q), np.percentile(xs, q*100))

0.05 estimate vs real 0.03576623657392189 0.0363860768755
0.5 estimate vs real 0.49868795164865265 0.499089434151
0.95 estimate vs real 0.9497762058615382 0.948161951597


*** One of the most valuable things about probabilistic data strucutre is that they kinda "invert" the modelling process, i.e., they allow the user to specify the error bounds, and do the calculation online for stream data. By this way, we don't have the "batch windows" anymore, i.e., we don't have to collect data, fit the model and do inference in batches from time to time.***